In [62]:
# Mengimpor library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk

# Mendownload daftar kata yang ada (vocabulary)
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
indo = stopwords.words('indonesian')

# Mengimpor dataset
dataset = pd.read_csv('training_gojek.csv')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anjas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
# Melakukan proses cleaning pada teks
corpus = []
for i in range(0, len(dataset)):
    #re = hapus / rubah
    review = re.sub('[^a-zA-Z]', ' ', dataset['komentar'][i])
    review = review.lower()
    review = review.split()    
    # Menghilangkan kata yang tidak ada di stopwords
    psi = StemmerFactory()
    ps = psi.create_stemmer()
    review = [ps.stem(word) for word in review if not word in indo]
    print(i,review)
    review = ' '.join(review)
    corpus.append(review)

0 ['kesal', 'driver', 'yg', 'pas', 'hujan', 'gak', 'ambil', 'order', 'udah', 'harga', 'syukur', 'yg', 'order', 'mah', 'diliatin', 'doang', 'gak', 'ambil', 'duduk', 'belah', 'ngeliatin', 'order', 'trus', 'pilih', 'yg', 'doang']
1 ['tolong', 'donk', 'daftar', 'akun', 'gojek', 'pakai', 'ktp', 'nomor', 'kk', 'sembarang', 'akun', 'gojek', 'anak', 'sekolah', 'pakai', 'nomor', 'induk', 'siswa', 'mikir', 'order', 'fiktif', 'sinkron', 'nomor', 'hp', 'mudah', 'mudah', 'order', 'fiktif', 'percaya']
2 ['hai', 'gojek', 'map', 'go', 'ride', 'rute', 'jalan', 'muter', 'tolong', 'baik', 'tindak', 'nya', 'ya', 'rugi']
3 ['layan', 'gosend', 'shopie', 'busuk', 'layan', 'bandara', 'soetta', 'parah', 'bawa', 'kode', 'pesan', 'langsung', 'tanganin', 'tugas', 'gocar', 'bilang', 'urut', 'nama', 'gk', 'catat', 'gk', 'tuju', 'belaga', 'sibuk', 'tlp', 'tlpn', 'menit', 'nama', 'gk', 'bilang', 'catat', 'du', 'ulas', 'lengkaplayanan', 'gosend', 'shopie', 'busuk', 'layan', 'bandara', 'soetta', 'parah', 'bawa', 'kode'

28 ['belum', 'mohon', 'maaf', 'singgung', 'pimpin', 'sangat', 'kecewa', 'lapor', 'isi', 'gopay', 'yg', 'blum', 'masuk', 'trima', 'tampa', 'respon', 'tanggal', 'desember', 'mohon', 'respon', 'resi', 'kirim', 'email', 'mohon', 'b', 'ulas', 'lengkapse', 'belum', 'mohon', 'maaf', 'singgung', 'pimpin', 'sangat', 'kecewa', 'lapor', 'isi', 'gopay', 'yg', 'blum', 'masuk', 'trima', 'tampa', 'respon', 'tanggal', 'desember', 'mohon', 'respon', 'resi', 'kirim', 'email', 'mohon', 'urus', 'pt', 'gojek', 'tindak', 'lanjut', 'terima', 'kasih']
29 ['super', 'gob', 'gk', 'masak', 'ganti', 'menu', 'drivernya', 'gimana', 'sistem', 'cancel', 'call', 'center', 'yg', 'cancel', 'driverr', 'pa', 'customer', 'restonya', 'enak', 'ganti', 'menu', 'free']
30 ['tolong', 'butuh', 'org', 'yg', 'nama', 'fotman', 'driver', 'gojek', 'batam', 'tolong', 'jumpa', 'udah', 'gk', 'sopan', 'mabuk', 'maki', 'maki', 'costemer', 'tolong', 'respon', 'pandang', 'jelek', 'gojek', 'karna', 'kmi', 'rame', 'info', 'kabarin']
31 ['kenda

56 ['argo', 'lonjak', 'utk', 'driver', 'kala', 'hujan', 'aplikasi', 'customer', 'argo', 'nya', 'tp', 'aplikasi', 'driver', 'argonya', 'gak', 'argo', 'lonjak', 'nya', 'telen', 'aplikator', 'driver', 'yg', 'basah', 'an', 'resiko', 'tp', 'gak', 'harga']
57 ['update', 'parah', 'pesan', 'goride', 'gofood', 'hp', 'update', 'update', 'normal', 'aja', 'pas', 'logout', 'login', 'hape', 'hp', 'gak', 'iya', 'sen', 'pinjem', 'hp', 'orang', 'ulas', 'lengkapupdate', 'parah', 'pesan', 'goride', 'gofood', 'hp', 'update', 'update', 'normal', 'aja', 'pas', 'logout', 'login', 'hape', 'hp', 'gak', 'iya', 'sen', 'pinjem', 'hp', 'orang', 'aneh']
58 ['cs', 'nya', 'senang', 'kali', 'telepon', 'kali', 'sibuk', 'twitter', 'dm', 'direspon', 'mention', 'respons', 'cust', 'direspon', 'mon', 'map', 'nggak', 'resprct']
59 ['guna', 'perhati', 'kabar', 'guna', 'pakai', 'gopay', 'mission', 'kali', 'kali', 'status', 'selesai', 'gopay', 'nya', 'suka', 'ganti', 'tunai', 'isi', 'gopay', 'promo', 'laku', 'guna', 'pengg', 'u

82 ['aplikasi', 'anjing', 'ngorder', 'tulis', 'server', 'sibuk', 'server', 'sibuk', 'babi', 'udah', 'uninstall', 'aja', 'ni', 'app', 'niat', 'order', 'go', 'food', 'ga', 'keburu', 'mati', 'lapar', 'gua', 'nya', 'anjing', 'kalo', 'gua', 'nyinggung', 'salah', 'lu', 'orang', 'laper', 'error', 'kesel', 'gua', 'kalo', 'kesel', 'toxic', 'ulas', 'lengkapaplikasi', 'anjing', 'ngorder', 'tulis', 'server', 'sibuk', 'server', 'sibuk', 'babi', 'udah', 'uninstall', 'aja', 'ni', 'app', 'niat', 'order', 'go', 'food', 'ga', 'keburu', 'mati', 'lapar', 'gua', 'nya', 'anjing', 'kalo', 'gua', 'nyinggung', 'salah', 'lu', 'orang', 'laper', 'error', 'kesel', 'gua', 'kalo', 'kesel', 'toxic', 'ga', 'tahan', 'sip', 'uninstall', 'ni', 'aplikasi', 'tolol', 'mari', 'pindah', 'grab', 'biaya', 'ongkir', 'nya', 'murah']
83 ['habis', 'kuota', 'gopay', 'yg', 'lanjur', 'beli', 'akn', 'lgsg', 'uninstall', 'aplikasi', 'setia', 'grab', 'komunikasi', 'keluh', 'tanggap', 'serius', 'saatt', 'jg', 'mundur', 'thanks']
84 ['go',

104 ['kasih', 'bintang', 'karna', 'pesan', 'go', 'jek', 'proses', 'cari', 'gojek', 'temu', 'gojek', 'pas', 'selesai', 'cari', 'eh', 'temu', 'pesan', 'karna', 'batal', 'ulang', 'kali', 'eh', 'beku']
105 ['app', 'versi', 'gak', 'guna', 'gak', 'blank', 'putih', 'uda', 'uninstall', 'install', 'playstore', 'gak', 'login', 'bufer', 'gak', 'notifikasi', 'sms', 'parah', 'banget', 'uninstall', 'cari', 'versi', 'nya', 'uda', 'install', 'login', 'notifikasi', 'sms', 'bgt', 'ulas', 'lengkapapp', 'versi', 'gak', 'guna', 'gak', 'blank', 'putih', 'uda', 'uninstall', 'install', 'playstore', 'gak', 'login', 'bufer', 'gak', 'notifikasi', 'sms', 'parah', 'banget', 'uninstall', 'cari', 'versi', 'nya', 'uda', 'install', 'login', 'notifikasi', 'sms', 'bgt', 'update', 'versi', 'playstore', 'aja', 'gak', 'blank', 'putih', 'revisi', 'versi', 'baru']
106 ['udh', 'ga', 'pakai', 'aplikasi', 'pesan', 'go', 'car', 'pakai', 'promo', 'dri', 'k', 'k', 'tolak', 'mentah', 'dri', 'drivernya', 'aplikasi', 'error', 'tpi', 

...

265 ['app', 'guna', 'mudah', 'gofood', 'istri', 'pakai', 'gofood', 'apalgi', 'pas', 'vocher', 'hari', 'biaa', 'x', 'pesan', 'gofood']
266 ['drivernya', 'sopan', 'ramah', 'hp', 'lowbat', 'posisi', 'butuh', 'banget', 'abang', 'tawar', 'pinjam', 'power', 'bank', 'nganter', 'go', 'food', 'cepat', 'acak', 'acak', 'tinggal', 'rombong', 'mesan', 'go', 'jek', 'jam', 'malam', 'gojek', 'percaya', 'ulas', 'lengkapdrivernya', 'sopan', 'ramah', 'hp', 'lowbat', 'posisi', 'butuh', 'banget', 'abang', 'tawar', 'pinjam', 'power', 'bank', 'nganter', 'go', 'food', 'cepat', 'acak', 'acak', 'tinggal', 'rombong', 'mesan', 'go', 'jek', 'jam', 'malam', 'gojek', 'percaya', 'tempat', 'lumayan', 'rawan', 'yaa', 'inti', 'gojek', 'percaya', 'layan', 'bintang', 'banget', 'trimakasih', 'gojek']
267 ['aplikasi', 'gojek', 'darurat', 'antar', 'mana', 'pergi', 'terimakasih', 'gojek', 'driver', 'gojek']
268 ['bantu', 'pas', 'kerja', 'qta', 'tinggal', 'pesan', 'gak', 'pkai', 'ribet', 'sikap', 'driver', 'nya', 'sll', 'ramah

292 ['bantu', 'pergi', 'beli', 'makan', 'overall', 'oke', 'neng', 'pake', 'gojek', 'drpd', 'grab', 'map', 'akurat', 'jarang', 'nyasar', 'moga', 'update', 'aplikasi', 'oke', 'vocer', 'potong', 'ongkir', 'harga', 'best', 'banget', 'gofood']
293 ['driver', 'd', 'mendapt', 'driver', 'yg', 'karna', 'driver', 'yg', 'sdh', 'tau', 'karakter', 'harga', 'order', 'langsung', 'd', 'terima', 'klau', 'judes', 'pelit', 'suka', 'mampir', 'kasih', 'tip', 'males', 'deh', 'terima', 'order', 'ulas', 'lengkapjika', 'driver', 'd', 'mendapt', 'driver', 'yg', 'karna', 'driver', 'yg', 'sdh', 'tau', 'karakter', 'harga', 'order', 'langsung', 'd', 'terima', 'klau', 'judes', 'pelit', 'suka', 'mampir', 'kasih', 'tip', 'males', 'deh', 'terima', 'order', 'curhatan', 'driver', 'yg', 'sy', 'd', 'untung', 'rugi', 'argo', 'd', 'tip', 'rb', 'murah', 'dri', 'argo', 'taksi']
294 ['bantu', 'saran', 'nih', 'tolong', 'tampilin', 'alas', 'driver', 'ngebatalin', 'order', 'sering', 'driver', 'udah', 'nerima', 'udah', 'chat', 'uda

323 ['min', 'sya', 'akun', 'gopay', 'muncul', 'aplikasi', 'gojek', 'driver', 'guna', 'gojek', 'driver', 'gopay']
324 ['transaksi', 'token', 'pln', 'kemarin', 'malam', 'kode', 'token', 'masuk', 'saldo', 'langsung', 'potong', 'hapus', 'ulas', 'kode', 'token', 'masuk']
325 ['aplikasi', 'zaman', 'now', 'aplikasi', 'utama', 'consumen', 'serba', 'yg', 'komplit', 'tinggal', 'pencet', 'butuh', 'penuh', 'mantap', 'dech', 'pokok', 'hidup', 'makarim']
326 ['tolong', 'gmn', 'biar', 'lokasi', 'tera', 'map', 'kasi', 'abang', 'drivernya', 'krn', 'titik', 'tdk', 'sesuai', 'ya', 'gak', 'sya', 'guna', 'berat', 'aplikasi', 'minimal', 'x', 'sya', 'nge', 'gofood', 'alamat', 'sya', 'sndri', 'ssh', 'cari', 'udah', 'ditolongin', 'dibenerin', 'ulas', 'lengkaptolong', 'gmn', 'biar', 'lokasi', 'tera', 'map', 'kasi', 'abang', 'drivernya', 'krn', 'titik', 'tdk', 'sesuai', 'ya', 'gak', 'sya', 'guna', 'berat', 'aplikasi', 'minimal', 'x', 'sya', 'nge', 'gofood', 'alamat', 'sya', 'sndri', 'ssh', 'cari', 'udah', 'ditol

348 ['tolong', 'tumpang', 'gojek', 'pake', 'hati', 'nurani', 'dikit', 'udah', 'ken', 'argo', 'murah', 'titik', 'kordinat', 'akalin', 'barang', 'bawa', 'bawa', 'anak', 'puter', 'driver', 'hidup', 'anak', 'istri', 'orang', 'tua', 'bayar', 'listrik', 'air', 'beli', 'bensin', 'services', 'motor', 'kalo', 'tumpang', 'ngelunj', 'ulas', 'lengkaptolong', 'tumpang', 'gojek', 'pake', 'hati', 'nurani', 'dikit', 'udah', 'ken', 'argo', 'murah', 'titik', 'kordinat', 'akalin', 'barang', 'bawa', 'bawa', 'anak', 'puter', 'driver', 'hidup', 'anak', 'istri', 'orang', 'tua', 'bayar', 'listrik', 'air', 'beli', 'bensin', 'services', 'motor', 'kalo', 'tumpang', 'ngelunjak', 'driver', 'muak', 'terimakasih']
349 ['cepat', 'dlm', 'jemput', 'tarip', 'jangkau', 'sayang', 'tdk', 'promo', 'paket', 'langgan', 'perlu', 'jalan']
350 ['alhamdulillah', 'gojek', 'online', 'bantu', 'orang', 'gak', 'kendara', 'salut', 'indonesia', 'aplikasi', 'on', 'line', 'kasih', 'bintang', 'terima', 'kasih', 'google']
351 ['gojek', 'emn

382 ['tolong', 'donk', 'aplikasi', 'biar', 'ngesave', 'lokasi', 'berangkat', 'tuju', 'rumah', 'kerja', 'kerja', 'rumah', 'biar', 'klik', 'oke', 'jalan']
383 ['dear', 'gojek', 'tolong', 'rate', 'driver', 'ubah', 'langgan', 'kasihan', 'driver', 'sengaja', 'salah', 'bintang', 'adil', 'komplain', 'pesan', 'gak', 'app', 'selesai', 'ulas', 'lengkapdear', 'gojek', 'tolong', 'rate', 'driver', 'ubah', 'langgan', 'kasihan', 'driver', 'sengaja', 'salah', 'bintang', 'adil', 'komplain', 'pesan', 'gak', 'app', 'selesai', 'pas', 'driver', 'cerita', 'gak', 'sengaja', 'selesai', 'pesan', 'nya', 'panik', 'bayar', 'pake', 'gopay', 'udah', 'terima', 'uang', 'nya', 'terima', 'kasih', 'driver', 'nya', 'ya', 'maaf']
384 ['bagus', 'muas', 'ajah', 'masuk', 'promo', 'mamimumemo', 'tinjau', 'harga', 'paket', 'ayam', 'nasi', 'lalap', 'ribu', 'mamimumemo', 'ribu', 'sekian', 'sih', 'nama', 'main', 'harga', 'rb', 'cocok', 'ga', 'rb', 'yg', 'telor', 'geprek', 'nasi', 'rb', 'harga', 'asli', 'segitu', 'g', 'ulas', 'len

In [73]:
#tf-idf


articles=np.array(corpus)
labels=np.array(dataset['sentimen'])

tf_vectorizer=TfidfVectorizer(min_df=4,max_df=0.3,ngram_range=(1,3))
x_train_tfidf=tf_vectorizer.fit_transform(articles).toarray()
x_test_tfidf=tf_vectorizer.transform(articles)

X_train, X_test, y_train, y_test=train_test_split(x_train_tfidf,labels,test_size=0.3,random_state = 0)


In [74]:
# Metode SVM
from sklearn.svm import SVC
classifierSVM = SVC(kernel = 'linear', random_state = 0)
classifierSVM.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [75]:
# Metode Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifierNB = GaussianNB()
classifierNB.fit(X_train, y_train)


GaussianNB(priors=None, var_smoothing=1e-09)

In [76]:
# Memprediksi hasil Test Set
y_pred_SVM = classifierSVM.predict(X_test)  # SVM
y_pred_NB = classifierNB.predict(X_test)    # Naive Bayes

In [77]:
from sklearn.metrics import accuracy_score,recall_score,precision_score
print("akurasi SVM = ", accuracy_score(y_test, y_pred_SVM, normalize=False))
print("akurasi NBC = ", accuracy_score(y_test, y_pred_NB, normalize=False))

print("recall SVM = ", recall_score(y_test, y_pred_SVM, average='micro'))
print("recall NBC = ", recall_score(y_test, y_pred_NB, average='micro'))

print("precision SVM = ", precision_score(y_test, y_pred_SVM, average='micro'))
print("precision NBC = ", precision_score(y_test, y_pred_NB, average='micro'))


akurasi SVM =  85
akurasi NBC =  84
recall SVM =  0.7083333333333334
recall NBC =  0.7
precision SVM =  0.7083333333333334
precision NBC =  0.7
